This kernel is a basic visualisation of correlation between pairwaise currencies. It has an emphasis on data visualisation informed by human perception research. Feel free to skip the words for the visualisations only if that's your interest. I appreciate any comments/suggestions and hope you enjoy.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import plotly
import plotly.plotly as py
import plotly.tools as plotly_tools
import plotly_express as px

In [2]:
df = pd.read_csv("currency_exchange_rates_02-01-1995_-_02-05-2018.csv", header=0)
df.head()

,Date,Algerian Dinar,Australian Dollar,Bahrain Dinar,Bolivar Fuerte,Botswana Pula,Brazilian Real,Brunei Dollar,Canadian Dollar,Chilean Peso,...,South African Rand,Sri Lanka Rupee,Swedish Krona,Swiss Franc,Thai Baht,Trinidad And Tobago Dollar,Tunisian Dinar,U.A.E. Dirham,U.K. Pound Sterling,U.S. Dollar
0,1995-1-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,1995-1-3,NaN,NaN,0.376,NaN,NaN,0.843,NaN,1.4035,NaN,...,3.55,50.04,7.4588,1.3168,25.10,5.8750,NaN,3.671,1.5617,1.0
2,1995-1-4,NaN,0.7704,0.376,NaN,NaN,0.844,NaN,1.4026,NaN,...,3.56,50.07,7.4480,1.3161,25.13,5.8738,NaN,3.671,1.5605,1.0
3,1995-1-5,NaN,0.7693,0.376,NaN,NaN,0.844,NaN,1.4002,NaN,...,3.55,50.07,7.4450,1.3080,25.14,5.8700,NaN,3.671,1.5610,1.0
4,1995-1-6,NaN,0.7699,0.376,NaN,NaN,0.842,NaN,1.4002,NaN,...,3.56,50.07,NaN,1.3040,25.13,5.8810,NaN,3.671,1.5645,1.0


In [3]:
# reduce data by half to reduce runtime
df.drop(df.index[0:len(df)/2], inplace =True)

In [4]:
# add integer ID for plotting marker later
df.insert(0, 'ID', range(1, (1+len(df))))

In [5]:
# df['Date'].dtype

In [6]:
# df['Date'] = pd.to_datetime(df['Date'])

Due to runtime limitation and screen estate, I selected top 10 and top 5 most traded currencies according to Triennial Central Bank Survey (2016). Except USD as it is the base currency, and HKD and Turkish Iira as they are not in this dataset. 


In [7]:
# include only top 10 currencies
df_top10 = df[['Date',
 'Euro',
 'Japanese Yen',
 'U.K. Pound Sterling',
 'Australian Dollar',
 'Canadian Dollar',
 'Swiss Franc',
 'Chinese Yuan',
 'Swedish Krona',
 'New Zealand Dollar',
 'Mexican Peso']]

# include only top 5 currencies
df_top5 = df[['ID','Date',
 'Euro',
 'Japanese Yen',
 'U.K. Pound Sterling',
 'Australian Dollar',
 'Canadian Dollar']]

In [ ]:
# Calculate correlation coefficient
allCorr = df.corr()
top10Corr = df_top10.corr()
top5Corr = df_top5.corr()


# Simple correlation heat map with Seaborn 


In [ ]:
# corr.style.background_gradient(cmap='coolwarm').set_properties(**{'font-size': '0pt'})
top10Corr.style.background_gradient(cmap='BrBG').set_precision(3)
cm = sns.choose_colorbrewer_palette('diverging', as_cmap=True)
graph = top10Corr.style.background_gradient(cmap=cm, axis=None).set_precision(3)
# By setting axis=None, it is now possible to compute the colors based on the entire matrix rather than per column or per row:
graphRendered = graph.render()
# if graph is not string then need to change it to string 
Html_file= open("diverging-graph.html","w")
Html_file.write(graphRendered)
Html_file.close()
graph

interactive(children=(Dropdown(description='name', options=('RdBu', 'RdGy', 'PRGn', 'PiYG', 'BrBG', 'RdYlBu', …

,Euro,Japanese Yen,U.K. Pound Sterling,Australian Dollar,Canadian Dollar,Swiss Franc,Chinese Yuan,Swedish Krona,New Zealand Dollar,Mexican Peso
Euro,1,-0.412,0.699,0.561,-0.8,0.104,0.206,-0.858,0.305,-0.813
Japanese Yen,-0.412,1,0.135,-0.71,0.619,0.4,0.336,0.466,-0.321,0.267
U.K. Pound Sterling,0.699,0.135,1,0.257,-0.574,0.467,0.538,-0.705,0.177,-0.843
Australian Dollar,0.561,-0.71,0.257,1,-0.862,-0.494,-0.318,-0.798,0.808,-0.521
Canadian Dollar,-0.8,0.619,-0.574,-0.862,1,0.0884,-0.00816,0.917,-0.578,0.825
Swiss Franc,0.104,0.4,0.467,-0.494,0.0884,1,0.875,0.101,-0.627,-0.316
Chinese Yuan,0.206,0.336,0.538,-0.318,-0.00816,0.875,1,-0.0346,-0.439,-0.326
Swedish Krona,-0.858,0.466,-0.705,-0.798,0.917,0.101,-0.0346,1,-0.618,0.848
New Zealand Dollar,0.305,-0.321,0.177,0.808,-0.578,-0.627,-0.439,-0.618,1,-0.255
Mexican Peso,-0.813,0.267,-0.843,-0.521,0.825,-0.316,-0.326,0.848,-0.255,1


Only the 10 selected currencies are displayed here due to large runtime for the entire dataset.

Diverging color scheme is used to show directionality and is checked for color-deficient vision. Here, each currency is a foreign-exchange pair with USD as denominator. Listed in the order of trading turnover value.

Nevertheless, we cannot solely rely on coefficients as the data patterns behind can vary and remain undetected by the coefficients. It is natural for us to use scatterplot for visualising pairwise relationships, which is also supported by previous research in infoVis.

## Some background about scatterplot:

Scatterplot can be perceived easily as it is comparing with a common scale along x- and y- axis [3]. 

In terms of perceptual correlation, Rensink et al. showed that human perceptions of correlational differences in scatterplot has a linear relationship with the objective correlational differences illustrated by Weber’s Law [2]. 

This was extended to a crowdsourced experiment that showed scatterplot is the most effective for human to perceive in both positive and negative correlations, whereas the similarly effective parallel coordinate only perform well in negative correlation [1]. This support using scatterplot matrix to show bi-directional correlation between currency pairs. 

Let's visualize correlation with some scatterplots!

In [ ]:
sns.pairplot(df_top5, kind="scatter", dropna = True, diag_kind="kde",
             markers=".",
             plot_kws=dict(s=80, linewidth=1))


/anaconda3/lib/python3.7/site-packages/statsmodels/nonparametric/kde.py:448: RuntimeWarning:

invalid value encountered in greater

/anaconda3/lib/python3.7/site-packages/statsmodels/nonparametric/kde.py:448: RuntimeWarning:

invalid value encountered in less



In this matrix, only five pairs are legibly shown because scatterplot consume a larger screen estate than the heat map previously. 

In an interactive web display though, size can be easily enlarged and reduced to focus on the information at a time that showing the top ten currencies will be feasible. Using plotly, we can easily make interactive graphs.

## Plotly for interactive scatterplot matrix 

In [ ]:
fig = px.scatter_matrix(df_top5, dimensions=['Euro','Japanese Yen','U.K. Pound Sterling','Australian Dollar','Canadian Dollar'], color='ID', 
                  color_continuous_scale=px.colors.sequential.matter,
                      size_max=1,
                       opacity=1,
                       title='Currency Correlation Scatterplot Matrix',
                  hover_name=['Date'],
                      width=1000,
                      height=1000)


# unclutter the plot by adding white border to markers
for t in fig.data:

    t.marker.line.width = 0.4
    t.marker.line.color = "white"
    t.marker.size = 4

fig

In [ ]:
# to generate html file of the plot
plotly.offline.plot(fig)

Time dimension is plotted with a sequential colour scheme: more recent data point is darker while older data point is lighter. 


There are occlusions due to the large number of data points. But it is not a big problem given that dot colour changes gradually in noticeable pattern. The plotly scatterplot interactivity also show tooltip as you hover, corresponding to the visual information-seeking mantra of 1) overview-first, 2)zoom-and-filter, 3)details-on-demand.


1. Lane Harrison, Fumeng Yang, Steven Franconeri, and Remco Chang. 2014. Ranking Visualizations of Correlation Using Weber’s Law. IEEE Transactions on Visualization and Computer Graphics 20, 12: 1943–1952.

2. Ronald A. Rensink and Gideon Baldridge. 2010. The Perception of Correlation in Scatterplots. Computer Graphics Forum 29, 3: 1203–1210.

3. Visualization Analysis and Design [Book]. Retrieved April 17, 2019 from https://www.oreilly.com/library/view/visualization-analysis-and/9781466508910/.